In [ ]:
# Tests for base Fault functionality: initialization, dumping, visualizations, etc.
%load_ext autoreload
%autoreload 2

import os
import sys
import warnings
import numpy as np

warnings.filterwarnings('ignore')

sys.path.insert(0, '../../../seismiqb')

from seismiqb import Field, Fault

In [ ]:
def compare_arrays(a, b):
    return all([(item[0] == item[1]).all() for item in zip(a, b)])

In [ ]:
""" You can manage cube and fault for the test:

CUBE_PATH : str
    Path to an existed seismic cube.
FAULT_PATH : str
    Path to an existed seismic fault.
"""
# Tests parameters
OUTPUT_DIR = './fault_test_files'

CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_cube.sgy')
FAULT_PATH = os.path.join(OUTPUT_DIR, 'test_fault')

# Visualization parameters
SCALE = 1
SHOW_FIGURES = True

# Initialization

In [ ]:
%%time
field = Field(CUBE_PATH)
fault = Fault(FAULT_PATH, field=field)

fault.show()

In [ ]:
%%time
# from_points init
new_fault = Fault(storage=fault.points, field=fault.field, name='tester')

assert (fault.points == new_fault.points).all(), f"`from_points` initialization test failed: original and initialized fault points are unequal"

In [ ]:
%%time
# from_object init
new_fault = Fault(storage={'sticks': fault.sticks}, field=fault.field, name='tester')

assert (fault.points == new_fault.points).all(), f"`from_objects` with 'sticks' initialization test failed: original and initialized fault points are unequal"

In [ ]:
%%time
# from_object init
new_fault = Fault(storage={'sticks': fault.sticks, 'points': fault.points}, field=fault.field, name='tester')

assert (fault.points == new_fault.points).all(), f"`from_objects` with 'sticks' and 'points' initialization test failed: original and initialized fault points are unequal"
assert (fault.sticks == new_fault.sticks), f"`from_objects` with 'sticks' and 'points' initialization test failed: original and initialized fault sticks are unequal"

# Storage convertation

In [ ]:
%%time
# from_object init

fault_from_points = Fault(storage={'points': fault.points}, field=fault.field)

assert not fault_from_points.has_component('sticks'), f"Fault created from 'points' has 'sticks'"
assert not fault_from_points.has_component('nodes'), f"Fault created from 'points' has 'nodes'"
assert fault_from_points.direction == 0, f"'direction' is wrong"

assert len(fault_from_points.sticks) > 0, f"'sticks' was not created from 'points'"

In [ ]:
%%time
# from_object init

fault_from_sticks = Fault(storage={'sticks': fault.sticks}, field=fault.field)

assert not fault_from_sticks.has_component('nodes'), f"Fault created from 'sticks' has 'nodes'"
assert not fault_from_sticks.has_component('points'), f"Fault created from 'sticks' has 'points'"
assert fault_from_sticks.direction == 0, f"'direction' is wrong"

assert len(fault_from_sticks.nodes) > 0, f"'nodes' was not created from 'sticks'"

In [ ]:
%%time
# from_object init

fault_from_nodes = Fault(storage={'nodes': fault.nodes, 'simplices': fault.simplices}, field=fault.field)

assert not fault_from_nodes.has_component('points'), f"Fault created from 'nodes'/'simplices' has 'points'"
assert not fault_from_nodes.has_component('sticks'), f"Fault created from 'nodes'/'simplices' has 'sticks'"

assert len(fault_from_nodes.points) > 0, f"'points' was not created from 'nodes' and 'simplices'"
assert fault_from_nodes.direction == 0, f"'direction' is not defined"

# Dump

In [ ]:
dump_path_npz = os.path.join(OUTPUT_DIR, 'tmp_fault.npz')

In [ ]:
%%time
# dump npz
fault.dump_npz(path=dump_path_npz)

dumped_fault = Fault(storage=dump_path_npz, field=field)

assert (fault.points == dumped_fault.points).all(), f"`dump_npz` test failed: original and dumped 'points' are unequal"
assert compare_arrays(fault.sticks, dumped_fault.sticks), f"`dump_npz` test failed: original and dumped 'sticks' are unequal"
assert (fault.nodes == dumped_fault.nodes).all(), f"`dump_npz` test failed: original and dumped 'nodes' are unequal"
assert (fault.simplices == dumped_fault.simplices).all(), f"`dump_npz` test failed: original and dumped 'simplices' are unequal"
assert (fault.direction == dumped_fault.direction), f"`dump_npz` test failed: original and dumped 'direction' are unequal"

os.remove(dump_path_npz)

In [ ]:
%%time
# dump fault sticks
dump_path_sticks = os.path.join(OUTPUT_DIR, 'tmp_fault.char')
fault.dump_fault_sticks(path=dump_path_sticks)

dumped_fault = Fault(storage=dump_path_sticks, field=field)

assert compare_arrays(fault.sticks, dumped_fault.sticks), f"`dump_fault_sticks` test failed: original and dumped 'sticks' are unequal"

os.remove(dump_path_sticks)

# Visualizations

In [ ]:
savepath = os.path.join(OUTPUT_DIR, 'tmp_fault_3d.html') 

fault.show_3d(sticks=False, show=SHOW_FIGURES, savepath=savepath)

os.remove(savepath)

In [ ]:
fault.show_3d(sticks=True, show=SHOW_FIGURES, savepath=savepath)

os.remove(savepath)

In [ ]:
savepath = os.path.join(OUTPUT_DIR, 'tmp_fault_2d.png') 

fault.show(show=SHOW_FIGURES, savepath=savepath)

os.remove(savepath)